# Import statements

In [ ]:
from urllib import request
import pandas as pd
from collections import Counter
import random
import numpy as np
import pandas as pd
import copy
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [ ]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [ ]:
from dont_patronize_me import DontPatronizeMe

In [ ]:
dpm = DontPatronizeMe('.', '.')

In [ ]:
dpm.load_task1()
dpm.load_task2(return_one_hot=True)

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


# Load data

In [ ]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)


In [ ]:
data=dpm.train_task1_df
data

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0
...,...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0
10466,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3
10467,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4


In [ ]:
rows = [] # will contain par_id, label and text
for idx in range(len(trids)):
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })

trdf1 = pd.DataFrame(rows)
trdf1

,par_id,community,text,label
0,4341,poor-families,"The scheme saw an estimated 150,000 children f...",1
1,4136,homeless,Durban 's homeless communities reconciliation ...,1
2,10352,poor-families,The next immediate problem that cropped up was...,1
3,8279,vulnerable,Far more important than the implications for t...,1
4,1164,poor-families,To strengthen child-sensitive social protectio...,1
...,...,...,...,...
8370,8380,refugee,Rescue teams search for survivors on the rubbl...,0
8371,8381,hopeless,The launch of ' Happy Birthday ' took place la...,0
8372,8382,homeless,"The unrest has left at least 20,000 people dea...",0
8373,8383,hopeless,You have to see it from my perspective . I may...,0


In [ ]:
rows = [] # will contain par_id, label and text
for idx in range(len(teids)):
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })
tedf1 = pd.DataFrame(rows)
tedf1

,par_id,community,text,label
0,4046,hopeless,We also know that they can benefit by receivin...,1
1,1279,refugee,Pope Francis washed and kissed the feet of Mus...,1
2,8330,refugee,Many refugees do n't want to be resettled anyw...,1
3,4063,in-need,"""Budding chefs , like """" Fred """" , """" Winston ...",1
4,4089,homeless,"""In a 90-degree view of his constituency , one...",1
...,...,...,...,...
2089,10462,homeless,"The sad spectacle , which occurred on Saturday...",0
2090,10463,refugee,""""""" The Pakistani police came to our house and...",0
2091,10464,disabled,"""When Marie O'Donoghue went looking for a spec...",0
2092,10465,women,"""Sri Lankan norms and culture inhibit women fr...",0


In [ ]:
rows_list = tedf1.values.tolist()

# Shuffle the list using random.shuffle
random.shuffle(rows_list)

# Create a new DataFrame from the shuffled list
shuffled_df = pd.DataFrame(rows_list, columns=tedf1.columns)
shuffled_df

,par_id,community,text,label
0,10078,immigrant,""""""" Previously there were lots of immigrants ,...",0
1,9013,disabled,""""""" We are a non-profit organisation that care...",0
2,8642,immigrant,The United Methodist Church where undocumented...,0
3,10028,poor-families,"However , the recent change in minimum wage ca...",0
4,10386,poor-families,"Kempeneer , who said his organisation had the ...",0
...,...,...,...,...
2089,10375,women,"Icon Singapore , a luxury lifestyle magazine f...",0
2090,8437,vulnerable,"New World Development ( NWD , 00017 . HK ) own...",0
2091,8508,migrant,""""""" If you look at patent data , for example ,...",0
2092,9077,disabled,Early Stimulation Programme for disabled child...,0


# Data analysis

## Analysis of the class labels
- how frequent these are
- how they correlate with any feature of the data, e.g. input length.

### Frequency

In [ ]:
data['orig_label'].value_counts().to_frame()

,orig_label
0,8529
1,947
3,458
4,391
2,144


In [ ]:
data['orig_label'].value_counts(normalize=True).to_frame()

,orig_label
0,0.814691
1,0.090458
3,0.043748
4,0.037348
2,0.013755


In [ ]:
data['label'].value_counts().to_frame()

,label
0,9476
1,993


In [ ]:
data['label'].value_counts(normalize=True).to_frame()

,label
0,0.905149
1,0.094851


In [ ]:
fig = px.pie(data, names='orig_label')
fig.update_layout(width=650, height=400 , template="none")
fig.show()


In [ ]:
fig = px.bar(df, x="orig_label", y="Count", color="orig_label", color_discrete_sequence=px.colors.qualitative.D3,
                   labels={"orig_label": "PCL Level", "keyword": "Keyword"})
fig.update_layout(width=650, height=400 , template="none")
fig.update_layout(yaxis_title='Number of paragraphs')
fig.show()


In [ ]:
fig = px.bar(df[~df['orig_label'].isin(["0", "1"])], x="orig_label", y="Count", color="orig_label", color_discrete_sequence=px.colors.qualitative.D3,
                   labels={"orig_label": "PCL Level", "keyword": "Keyword"})
fig.update_layout(width=650, height=400 , template="none")
fig.update_layout(yaxis_title='Number of paragraphs')
fig.show()

### PCL label per keyword

In [ ]:
df = data.copy()
df['Count'] = 1

In [ ]:
fig = px.histogram(df, x="keyword", y="Count", color="orig_label", barmode="group", color_discrete_sequence=px.colors.qualitative.D3,
                   labels={"orig_label": "PCL Level", "keyword": "Keyword"})
fig.update_layout(width=650, height=400 , template="none")
fig.update_layout(yaxis_title='Number of paragraphs')
fig.show()


### PCL label per country

In [ ]:
fig = px.histogram(df, x="country", y="Count", color="orig_label", barmode="group", color_discrete_sequence=px.colors.qualitative.D3,
                   labels={"orig_label": "PCL Level", "country": "Country"})
fig.update_layout(width=650, height=400 , template="none")
fig.update_layout(yaxis_title='Number of paragraphs')
fig.show()


### Correlation with length

In [ ]:
df['length'] = df['text'].map(len)
df

,par_id,art_id,keyword,country,text,label,orig_label,Count,length
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0,1,620
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0,1,237
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0,1,162
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0,1,162
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0,1,277
...,...,...,...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1,1,348
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0,1,258
10466,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3,1,286
10467,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4,1,502


In [ ]:
df.corr()

<ipython-input-62-2f6f6606aa2c>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



,label,Count,length
label,1.000000,NaN,0.042174
Count,NaN,NaN,NaN
length,0.042174,NaN,1.000000


In [ ]:
fig = px.box(df, x="label", y="length", labels={"label": "Contrains PCL?", "length": "Length"})
fig.update_layout(width=650, height=400 , template="none")
fig.show()